In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize

In [37]:
data = pd.read_csv('../Additional_eurecom_code/Event_extraction/data/joined_train.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../Additional_eurecom_code/Event_extraction/data/joined_train.csv'

In [3]:
data = data.drop(data.columns[[0, 1]], axis=1)

In [ ]:
data = data[data['tag'].str.contains('effect')].reset_index(drop=True) #Only keep entries that have an effect and thus a relation
data.head(20)

In [21]:
"""
assumtpions:
- One subject and one object
- Every line has an effect
"""

data = data[data['tag'].str.contains('effect')] #Only keep entries that have an effect and thus a relation
triplets = []
labels = []
context = []
skipped = 0

for index, row in data.iterrows():

    sub_obj = {}

    tokenize_text = word_tokenize(row['text'])
    split_tags = row['tag'].split()

    if len(tokenize_text) != len(split_tags): #This means the tags and tokens do not overlap correctly, hence results are wrong.
        skipped += 1
        continue

    for word, tag in zip(tokenize_text, split_tags):

        if tag != '0': #an entity is detected

            if tag in sub_obj: #This means that the entity consists of 2 words
                sub_obj[tag] += f' {word}'
            else:
                sub_obj[tag] = word

    for key, value in sub_obj.items():
        if key != 'effect': #all other relations are the subjects
            subject = value
        else: #Effect is always the object
            object = value

    triplets.append(f"<triplet> {subject} <subj> {object} <obj> {row['label']}")
    labels.append(row['label'])
    context.append(row['text'])

final_data = pd.DataFrame(list(zip(range(len(triplets)), range(len(triplets)), context, triplets)),
                          columns = ['id', 'title', 'context', 'triplets'])

final_data.to_csv('Data/rebel_format.csv', index=False)
print(f"Number of skipped: {skipped}")

TypeError: unsupported operand type(s) for +=: 'spacy.tokens.token.Token' and 'str'

In [ ]:
# to get some statistics
num_zero = 0
num_no_effect = 0
index = 0
for line, label in zip(data['tag'], data['label']):
    flag = False

    if label == '0':
        num_zero +=1
        continue

    for tag in line.split():
        if tag == 'effect':
            flag = True
            break

    if flag == False:
        num_no_effect +=1
        print(f"{index} no effect")

    index +=1
print(f"number of no relations: {num_zero}")
print(f"number of no effects: {num_no_effect}")

In [4]:
#Used for data that is already split into trigger1, trigger2 and label
def convert_to_rebel(data):
    data = data[data.label != str(0)]
    data = data.rename(columns={"sentence": "context"})
    data['triplets'] = '<triplet> ' + data['trigger1'] + ' <subj> ' + data['trigger2'] + ' <obj> ' + data['label'] #Add the suitable format
    data = data.drop(['trigger1', 'trigger2', 'label'], axis=1)
    return data

In [42]:
#Used for mapping the updated file where data is already split
data = pd.read_csv('Data/rebel_v1/relation_data_updated.csv')
data= data.drop(data.columns[[0, 1, 2, 6, 8]], axis=1)
data = convert_to_rebel(data)
data.to_csv('Data/rebel_v2/rebel_format_v2.csv', index=False)

In [2]:
#Creating testset, using the non-synthetic sets, and checking if they occur in the train set of rebel
from sklearn.model_selection import train_test_split

enables = pd.read_csv('Data/rebel_v2/data/enables_stud.csv').drop(['verb'], axis=1)
enables = enables.rename(columns={"sentece": "sentence", "trigger_1": "trigger1", "trigger_2": "trigger2"})
intends = pd.read_csv('Data/rebel_v2/data/intend_left.csv')
intends['sentence'] = intends['sentence'].apply(lambda x: x[1:] if x.startswith(',') else x)
intends['sentence'] = intends['sentence'].apply(lambda x: x[2:] if x.startswith(' ,') else x)
prevents = pd.read_csv('Data/rebel_v2/data/prevents_stud.csv').drop(['verb'], axis=1)
prevents = prevents.rename(columns={"sentece": "sentence", "trigger_1": "trigger1", "trigger_2": "trigger2"})
causes = pd.read_csv('Data/rebel_v2/data/our_data.csv') #This also contains more than just cause
causes = causes.rename(columns={"sentece": "sentence", "trigger_1": "trigger1", "trigger_2": "trigger2"})

total = pd.concat([enables, intends, prevents, causes]).reset_index()
total = total[total['label'] != '0']
total = total.drop(['index', 'Unnamed: 0.1', 'Unnamed: 0', 'tag'], axis=1)
total.to_csv('Data/rebel_v2/data/rebel_merged_non_synthetic.csv')

rest_data, test_data = train_test_split(total, test_size=0.15, random_state=1, shuffle=True)
test_data.to_csv('Data/rebel_v2/data/new_split/test.csv')

In [4]:
enables = pd.read_csv('Data/rebel_v2/data/enables.csv')
intends = pd.read_csv('Data/rebel_v2/data/intends.csv')
prevents = pd.read_csv('Data/rebel_v2/data/prevents.csv')
#Cause is the same dataset as loaded above
#causes = causes.rename(columns={"sentece": "sentence", "trigger_1": "trigger1", "trigger_2": "trigger2"})

total = pd.concat([rest_data, enables, intends, prevents]).reset_index(drop=True)
total = total.drop(['Unnamed: 0.1', 'Unnamed: 0', 'tag'], axis= 1).drop_duplicates()
total = total[total['label'] != '0']

total.to_csv('Data/rebel_v2/data/new_split/train_val_combined.csv')
train_data, val_data = train_test_split(total, test_size=0.1, shuffle=True)
train_data.to_csv('Data/rebel_v2/data/new_split/train.csv')
val_data.to_csv('Data/rebel_v2/data/new_split/val.csv')

In [10]:
train_data = pd.read_csv('Data/rebel_v2/data/new_split/train.csv').drop(['Unnamed: 0'], axis=1)
val_data = pd.read_csv('Data/rebel_v2/data/new_split/val.csv').drop(['Unnamed: 0'], axis=1)
test_data = pd.read_csv('Data/rebel_v2/data/new_split/test.csv').drop(['Unnamed: 0'], axis=1)

train_data = convert_to_rebel(train_data)
val_data = convert_to_rebel(val_data)
test_data = convert_to_rebel(test_data)

train_data.to_csv('Data/rebel_v2/data/new_split/train_rebel.csv')
val_data.to_csv('Data/rebel_v2/data/new_split/val_rebel.csv')
test_data.to_csv('Data/rebel_v2/data/new_split/test_rebel.csv')

In [12]:
import pandas as pd
data=pd.read_csv('Data/rebel_v2/data/new_split/train_val_combined.csv').drop(data.columns[[0]], axis=1)
data['triplets'] = '<triplet> ' + data['trigger1'] + ' <subj> ' + data['trigger2'] + ' <obj> ' + data['label'] #Add the suitable format
data = data.sample(frac=1).reset_index(drop=True)
data.to_csv('Data/rebel_v2/data/new_split/train_val_combined_rebel.csv')